In [3]:
import pandas as pd
from graphs.plotly_graphs import *
from components.standard_components import get_growth
from data_proccessing.utils import generate_location_df

In [4]:
df = pd.read_excel("data/full_iseller_data.xlsx")

In [5]:
df['closed_date'] = pd.to_datetime(df['closed_date'])
df['date'] = df['closed_date'] .dt.date
df['day_of_week_num'] = df['closed_date'].dt.dayofweek
df['day_name'] = df['closed_date'].dt.day_name()


## Utils


In [5]:
def generate_sales_table(data):
    # Calculate current and last periods
    # df = pd.DataFrame(data)
    df = data.copy()

    current_quarter = df["quarter"].max()
    last_quarter = current_quarter - 1
    current_month = df["month"].max()
    last_month = current_month - 1

    # Filter dataframe once for each period
    df_current_quarter = df[df['quarter'] == current_quarter]
    df_last_quarter = df[df['quarter'] == last_quarter]
    df_current_month = df[df['month'] == current_month]
    df_last_month = df[df['month'] == last_month]

    # Groupby operation for each period
    total_revenue_df = df.groupby(['clean_product_name', 'product_type'])['total_order_amount'].sum().reset_index()
    total_revenue_df = total_revenue_df.rename(columns={'total_order_amount': 'total_revenue'})

    current_quarter_revenue = df_current_quarter.groupby(['clean_product_name', 'product_type'])['total_order_amount']\
        .sum().reset_index().rename(columns={'total_order_amount': 'current_quarter_revenue'})

    last_quarter_revenue = df_last_quarter.groupby(['clean_product_name', 'product_type'])['total_order_amount']\
        .sum().reset_index().rename(columns={'total_order_amount': 'last_quarter_revenue'})

    current_month_revenue = df_current_month.groupby(['clean_product_name', 'product_type'])['total_order_amount']\
        .sum().reset_index().rename(columns={'total_order_amount': 'current_month_revenue'})

    last_month_revenue = df_last_month.groupby(['clean_product_name', 'product_type'])['total_order_amount']\
        .sum().reset_index().rename(columns={'total_order_amount': 'last_month_revenue'})

    # Merge all dataframes into a single dataframe
    sales_performance_df = total_revenue_df.merge(current_quarter_revenue, on=["clean_product_name", "product_type"], how='left')\
        .merge(last_quarter_revenue, on=["clean_product_name", "product_type"], how='left')\
        .merge(current_month_revenue, on=["clean_product_name", "product_type"], how='left')\
        .merge(last_month_revenue, on=["clean_product_name", "product_type"], how='left')

    # Fill NaN values with 0 (if appropriate for your context)
    sales_performance_df = sales_performance_df.fillna(0)

    
    sales_performance_df['quarterly_growth'] = (sales_performance_df['current_quarter_revenue'] - sales_performance_df['last_quarter_revenue']) / sales_performance_df['last_quarter_revenue'] * 100
    sales_performance_df['monthly_growth'] = (sales_performance_df['current_month_revenue'] - sales_performance_df['last_month_revenue']) / sales_performance_df['last_month_revenue'] * 100
    
    # Calc growth
    sales_performance_df['quarter_Growth(%)'] = ((sales_performance_df['current_quarter_revenue'] - \
                                         sales_performance_df['last_quarter_revenue']) / (sales_performance_df['current_quarter_revenue'] +  sales_performance_df['last_quarter_revenue'])).round(2)
    sales_performance_df['Monthly_Growth(%)'] = ((sales_performance_df['current_month_revenue'] - \
                                            sales_performance_df['last_month_revenue']) / (sales_performance_df['current_month_revenue'] + sales_performance_df['last_month_revenue'])).round(2)
    return sales_performance_df

In [13]:
def generate_location_df(data):
    df = data.copy()
    # df = df[df['outlet_name'].isin(['Bekasi', 'Jaksel', 'Cibubur', 'Jakut', 'Tangerang', 'Bandung' ])]
    sales_loc_df = df.groupby(["outlet_name", "year", "month_name", "month"])["quantity"].sum().reset_index()
    sales_loc_df = sales_loc_df.sort_values(by=["year", "month", "quantity"], ascending=[True, True, False])
    sales_loc_df = sales_loc_df.sort_values(by=['outlet_name', 'year', 'month'])

    # Step 2: Calculate the difference in quantity between consecutive months for each location
    sales_loc_df['quantity_diff'] = sales_loc_df.groupby('outlet_name')['quantity'].diff()

    # Step 3: Calculate the growth rate
    sales_loc_df['growth_rate'] = (sales_loc_df['quantity_diff'] / sales_loc_df['quantity'].shift()) * 100
    return sales_loc_df

In [6]:
df['total_order_amount']

0          15001
1          25000
2              1
3              1
4              1
          ...   
62928    4968000
62929    5184000
62930    5831000
62931    5831000
62932    6168000
Name: total_order_amount, Length: 62933, dtype: int64

## Overview Page

### Prompts
* Which products have the highest and lowest sales quantities?

Identifying top and bottom sellers helps understand which products are most and least popular with customers, which can guide inventory and promotional strategies.

* How does the average price of the product affect the quantity sold?

* How much sales contribution does the top 10 products make?


In [14]:
# Top Product by Quantity Sold

df.groupby(['clean_product_name', 'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])

,clean_product_name,year,month,week,total_amount,quantity
3595,kawa kawa anggur hijau,2024,1,1,1.401732e+05,557
340,anggur merah gold,2024,1,1,1.314846e+05,395
457,api anggur hijau,2024,1,1,1.643224e+05,292
296,anggur merah,2024,1,1,1.186567e+05,231
1125,bintang kaleng,2024,1,1,2.227357e+05,224
...,...,...,...,...,...,...
989,baileys irish cream,2024,5,19,4.845000e+05,1
4533,paket cham yogurt,2024,5,19,2.004030e+05,1
4292,paket 1 smirnoff vodka green apple + 4 sprite...,2024,5,19,2.880170e+05,1
4344,paket 2 vibe regular + free 2 sloki vibe,2024,5,19,5.000000e+05,1


### Prompts
Same as above but for category instead


In [13]:
# Top Product by Category Sold
# Get average price of category too 

df.groupby(['product_type',  'year', 'month', 'week']).agg({'total_amount': 'mean', 'quantity': 'sum'}).reset_index()\
        .sort_values(by='quantity', ascending=False).sort_values(by=["year", "month", "week"])

,product_type,year,month,week,total_amount,quantity
0,ANGGUR,2024,1,1,1.371487e+05,1945
22,BEER,2024,1,1,1.907702e+05,1626
187,REGULAR,2024,1,1,2.065521e+05,645
253,SOJU,2024,1,1,2.468859e+05,502
321,VODKA,2024,1,1,2.238414e+05,422
...,...,...,...,...,...,...
163,MAKGEOLLI,2024,5,19,2.245525e+05,14
185,MERCHANDISE,2024,5,19,2.056200e+05,14
316,TEQUILA,2024,5,19,4.743783e+05,3
65,COGNAC,2024,5,19,1.188174e+06,2


* What are the monthly and yearly sales trends for each product?

This helps in understanding if sales are increasing, decreasing, or remaining stable over time, which is critical for forecasting and planning.
* Are there any noticeable seasonal patterns in the sales data?

Identifying peaks and troughs in sales can help in planning for seasonal demand variations, ensuring sufficient stock levels during high-demand periods.
Which months show the highest and lowest sales for each product?

This information is useful for promotional planning, inventory management, and identifying off-peak times that may need strategic initiatives to boost sales.
* How does the performance of different products compare across the same time periods (e.g., year-over-year comparisons for the same month)?

Comparing performance across the same periods in different years can reveal growth trends, the impact of external factors (like economic conditions or competition), and effectiveness of past strategies.
* Are there any emerging trends or shifts in consumer preferences over the months and years?

Spotting emerging trends or shifts can help in adjusting product offerings, marketing strategies, and inventory management to align with changing consumer preferences.

* What is the correlation between external events (such as holidays, festivals, or economic changes) and monthly sales patterns? (Move to forecasting)

Understanding the impact of external events can help in planning marketing campaigns and inventory to capitalize on these opportunities.
How do monthly sales figures correlate with marketing campaigns or promotions?

Analyzing the impact of marketing efforts on sales can guide future promotional strategies to maximize their effectiveness.

In [19]:
# Total Quantity sold each month
df.groupby(["year", "month", "clean_product_name"])['quantity'].sum().reset_index().sort_values(by=["year", "month"])


,year,month,clean_product_name,quantity
0,2024,1,(es batu) crystalline cup,955
1,2024,1,a&w root beer kaleng,81
2,2024,1,absolut vodka,32
3,2024,1,alexis anggur hijau,697
4,2024,1,alexis anggur merah,283
...,...,...,...,...
1883,2024,5,wija soju original,25
1884,2024,5,wija soju peach,12
1885,2024,5,wine opener stainless,1
1886,2024,5,yakult 1 pack,7


### Which products generate the highest and lowest revenue?

Identifying top and bottom revenue generators helps prioritize marketing efforts, inventory management, and product development.
Which products are the most and least profitable?

Focusing on profitability rather than just revenue can ensure that the business prioritizes products that contribute the most to the bottom line.
What is the profit margin for each product?

Calculating the profit margin (profit as a percentage of revenue) for each product helps identify which products are more efficient in generating profit relative to their sales.
Are there any products with high revenue but low profit margins?

Identifying such products can highlight opportunities to optimize costs, adjust pricing, or reconsider the product's strategic importance.
How do revenue and profit trends compare over time for each product?

Analyzing trends over time can help identify products that are growing in both revenue and profitability, as well as those that may be declining or facing challenges.
Which products contribute the most to overall company revenue and profit?

Understanding the contribution of each product to the total revenue and profit helps in resource allocation, strategic planning, and identifying key areas of focus.
How does the profitability of different product categories compare?

Comparing profitability across categories can reveal which segments of the business are most lucrative and which may need strategic adjustments.

In [17]:
# Total Quantity sold each Day
df.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'clean_product_name'])['quantity'].sum().reset_index().sort_values(by="day_of_week_num")


,year,month,week,day_name,day_of_week_num,clean_product_name,quantity
14481,2024,4,15,Monday,0,cointreau,1
13551,2024,4,14,Monday,0,jw black label,4
13552,2024,4,14,Monday,0,jw red label,3
13553,2024,4,14,Monday,0,kawa kawa anggur hijau,24
13554,2024,4,14,Monday,0,kawa kawa anggur merah,6
...,...,...,...,...,...,...,...
2786,2024,1,3,Sunday,6,gilbeys distiler's blend whisky,3
2787,2024,1,3,Sunday,6,gilbeys gin,3
2788,2024,1,3,Sunday,6,gilbeys vodka,2
2790,2024,1,3,Sunday,6,grey goose,2


# Sales Page

* Which products generate the highest and lowest revenue?

* Identifying top and bottom revenue generators helps prioritize marketing efforts, inventory management, and product development.
Which products are the most and least profitable?

* Focusing on profitability rather than just revenue can ensure that the business prioritizes products that contribute the most to the bottom line.
What is the profit margin for each product?

Calculating the profit margin (profit as a percentage of revenue) for each product helps identify which products are more efficient in generating profit relative to their sales.
* Are there any products with high revenue but low profit margins?

Identifying such products can highlight opportunities to optimize costs, adjust pricing, or reconsider the product's strategic importance.
* How do revenue and profit trends compare over time for each product?

Analyzing trends over time can help identify products that are growing in both revenue and profitability, as well as those that may be declining or facing challenges.
* Which products contribute the most to overall company revenue and profit?

Understanding the contribution of each product to the total revenue and profit helps in resource allocation, strategic planning, and identifying key areas of focus.
* How does the profitability of different product categories compare?

* Comparing profitability across categories can reveal which segments of the business are most lucrative and which may need strategic adjustments.
What are the trends in revenue and profit during promotional periods?

* Analyzing the impact of promotions on both revenue and profit helps determine the effectiveness of marketing campaigns and whether they provide sustainable value.
How do external factors (such as seasonality, economic conditions, or competitive actions) impact revenue and profit?

* Understanding external influences on financial performance can help in planning and mitigating risks associated with these factors.
Are there any emerging products with significant growth potential in revenue and profit?

* Identifying emerging products with high growth potential allows the business to capitalize on new opportunities and allocate resources effectively to support their growth.

In [25]:
### Sales table
sales_table = generate_sales_table(df)
sales_table.head()

,clean_product_name,product_type,total_revenue,current_quarter_revenue,last_quarter_revenue,current_month_revenue,last_month_revenue,quarterly_growth,monthly_growth,quarter_Growth(%),Monthly_Growth(%)
0,(es batu) crystalline cup,REGULAR,201033098,55569433.0,145463665.0,10521000.0,45048433.0,-61.798410,-76.645137,-0.45,-0.62
1,a&w root beer kaleng,REGULAR,21235100,5086300.0,16148800.0,89000.0,4997300.0,-68.503542,-98.219038,-0.52,-0.97
2,absolut vodka,VODKA,51599900,18173000.0,33426900.0,2180000.0,15993000.0,-45.633606,-86.369036,-0.30,-0.76
3,alexis anggur hijau,ANGGUR,182694900,40367800.0,142327100.0,9386500.0,30981300.0,-71.637306,-69.702692,-0.56,-0.53
4,alexis anggur merah,ANGGUR,70802100,18252100.0,52550000.0,2918100.0,15334000.0,-65.267174,-80.969740,-0.48,-0.68


In [6]:
# Top Product by profit
product_quantity = df.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'clean_product_name'])['profit'].sum().reset_index()\
        .sort_values(by='profit', ascending=False)
product_quantity.head()

,year,month,week,day_name,day_of_week_num,clean_product_name,profit
4931,2024,2,5,Friday,4,bintang botol,18341206.0
1204,2024,1,2,Friday,4,happy soju original,17331836.0
1305,2024,1,2,Friday,4,wija soju lychee,15659100.0
16271,2024,4,17,Friday,4,anggur merah gold,12750050.0
5219,2024,2,5,Saturday,5,kawa kawa anggur hijau,11733007.0


In [7]:
# Top Category by Profit
product_quantity = df.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'product_type'])['profit'].sum().reset_index()\
        .sort_values(by='profit', ascending=False)
product_quantity.head()

,year,month,week,day_name,day_of_week_num,product_type,profit
523,2024,2,5,Saturday,5,SOJU,6.352062e+07
117,2024,1,2,Friday,4,SOJU,5.664071e+07
1739,2024,4,16,Wednesday,2,BEER,4.875298e+07
562,2024,2,6,Friday,4,BEER,4.746571e+07
514,2024,2,5,Saturday,5,BEER,4.721337e+07


In [9]:
# Top Product by Revenue
product_quantity = df.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'clean_product_name'])['total_amount'].sum().reset_index()\
        .sort_values(by='total_amount', ascending=False)

# Top Category by Revenue
product_quantity = df.groupby(['year', 'month', 'week', 'day_name', 'day_of_week_num', 'product_type'])['total_amount'].sum().reset_index()\
        .sort_values(by='total_amount', ascending=False)

### Recheck generate_location_df() function

In [14]:
# Generate Location by Growth and Quantity
location_data = generate_location_df(df)
location_data.head()

,outlet_name,year,month_name,month,quantity,quantity_diff,growth_rate
1,Ayam! Alun-Alun BKS,2024,March,3,172,NaN,NaN
0,Ayam! Alun-Alun BKS,2024,April,4,134,-38.0,-22.093023
2,Ayam! Alun-Alun BKS,2024,May,5,23,-111.0,-82.835821
5,Ayam! Dapur Kitchen BKS,2024,January,1,366,NaN,NaN
4,Ayam! Dapur Kitchen BKS,2024,February,2,331,-35.0,-9.562842


In [45]:
# Weekly patterns for each month
weekly_sales_pattern = df.groupby(["month", "month_name", "week_of_month"])['quantity'].sum().reset_index()
weekly_sales_pattern.head()


,month,month_name,week_of_month,quantity
0,1,January,1,6597
1,1,January,2,6022
2,1,January,3,6099
3,1,January,4,6025
4,1,January,5,2134


In [47]:
# Daily sales pattern for each hour
daily_hour_pattern = df.groupby(["day_of_week_num", "day_name", "hour"])['quantity'].sum().reset_index()


## KPI Metric

* Modify function to get KPI for last 30 days
* Modify get_groth function to get growth vs last 30 days as opposed to last month

In [29]:
total_unit_sold = df['quantity'].sum()
total_revenue = df['total_order_amount'].sum()
total_profit = df['profit'].sum()
total_unique_customers = df["customer_id"].nunique()
total_orders = df.groupby('order_id').size().count()
total_unique_customers = df.groupby('customer_id').size().count()
avg_spending = df["total_order_amount"].mean().round(2)


unit_growth = get_growth(df, "month", 3, "quantity")
rev_growth = get_growth(df, "month", 3, "total_order_amount")
profit_growth = get_growth(df, "month", 3, "profit")
cust_growth = get_growth(df, "month", 3, "quantity")
order_growth = get_growth(df, "month", 3, "order_id" , "count")
avg_spending_growth = get_growth(df, "month", 3, "total_order_amount", "mean")

In [49]:
# Monthly revenue and profit growth pattern
monthly_data = df.groupby(['month']).agg({'total_amount': 'sum', 'profit': 'sum'}).reset_index().sort_values(by="month")
monthly_data.dropna(inplace=True)

monthly_data['revenue_growth_pct'] = monthly_data['total_amount'].pct_change() * 100
monthly_data['profit_growth_pct'] = monthly_data['profit'].pct_change() * 100

# Calculate month-over-month growth percentage for each product for both revenue and profit
monthly_data.dropna(inplace=True)
monthly_data

,month,total_amount,profit,revenue_growth_pct,profit_growth_pct
1,2,4361111708,2.899853e+09,5.129185,9.705985
2,3,3239331731,2.111685e+09,-25.722340,-27.179591
3,4,3301849103,2.115140e+09,1.929947,0.163653
4,5,1015196031,6.872488e+08,-69.253712,-67.508124


In [50]:
# Daily average spending over time
daily_average = df.groupby(['date', 'order_id'])['total_order_amount'].sum().reset_index()

# Group by month to get the average of these summed amounts
daily_avg = daily_average.groupby('date')['total_order_amount'].mean().reset_index()

# Rename the columns for clarity
daily_avg.columns = ['date', 'average_order_amount']
daily_avg

,date,average_order_amount
0,2024-01-01,257514.435146
1,2024-01-02,257363.487738
2,2024-01-03,283168.091168
3,2024-01-04,332737.215190
4,2024-01-05,368489.196464
...,...,...
125,2024-05-05,217041.414141
126,2024-05-06,211662.797342
127,2024-05-07,441440.869565
128,2024-05-08,294948.915187
